In [ ]:
%run -i "../Retropy_framework.py"

# Regression for UBT and TMF - x2 and x3 daily leveraged TLT

findings:
- lrret with rebal=none on pred is the worst, as is expected regressing a daily leveraged ETF
- lrret with fit_values=True has a better fit (R^2) overall, vs fit_values=False (returns), but on the other hand, the fit on returns is very smooth (indicating consistent regression) while on values is not.
- from this we gather that a returns regression with rebal=day is the best way to regress, and we should add a bias to compensate for any systemic differences in TLT/UBT
- another key note is that i_ret(ret(x)) does a much better fit than i_logret(logret(x)) in "TR" but worse in "PR", it seems to in general produce smaller returns than i_logret(logret). What does this mean about lrret in general?
- as expected the daily regression factor is not exactly 2.00 but rather 1.9529, indicating inefficiency in UBT
- regressing TR overshoots the base, PR undershoots, and NTR comes closest. Note sure why they don't all regress in a similar way.


notes:
- the smoothest regression is x2 daily returns of TLT, as that the the definition of UBT (x3 for TMF)
- we can add some bias to the daily return for a better fit, but it seems that the efficency of TLT/UBT changed over times, which makes the fixed bias unable to provide an exact fit in performance across all time periods
- there are slight changes in how a leveraged series looks when using i_ret(ret(x)) vs i_logret(logret(x)). It'n unclear yet which is the correct one to use. It seems that i_ret(ret(x)) provides a smoother relative regression, indicating that (upto efficiency of leveraged ETFs) it is the correct one to use

In [ ]:
mode = "PR" # try TR NTR PR

base = get(TLT, mode=mode)
target = get(UBT, mode=mode)
factor = 2
lr_factor = 1.9529
plus = 0 # 0.00011

target_ret = name(i_ret(ret(base)*factor), 'ret')
target_ret_as_lrret = name(i_ret(ret(base)*lr_factor+plus), 'ret_1.9529')
target_ret_as_lrret_get = get({base.name: lr_factor*100}, rebal="day", mode=mode)
target_logret = name(i_logret(logret(base)*factor), 'logret')
target_logret_as_lrret = name(i_logret(logret(base)*lr_factor), 'logret_1.9529')

args = dict(sum1=False, return_pred=True, show_res=False)
target_lrret_vals_none = name(get(lrret(target, [base], **args), rebal='none', mode=mode), 'lrret-vals-none')
target_lrret_vals_day  = name(get(lrret(target, [base], **args), rebal='day', mode=mode), 'lrrret-vals-day')
target_lrret_rets_none = name(get(lrret(target, [base], **args, fit_values=False), rebal='none', mode=mode), 'lrrret-rets-none')
target_lrret_rets_day  = name(get(lrret(target, [base], **args, fit_values=False), rebal='day', mode=mode), 'lrrret-rets-day')

all = [target, target_ret, target_ret_as_lrret, target_logret, target_logret_as_lrret, target_lrret_vals_none, target_lrret_vals_day, target_lrret_rets_none, target_lrret_rets_day, target_ret_as_lrret_get]
show(sdiv(all, target))



In [ ]:
show_dd(target_ret_as_lrret_get, target, mode="PR")
show(target_ret_as_lrret_get, target, trim=False)